# Cars: Getting Started

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import pyblp
sns.set_theme()
import matplotlib.pyplot as plt

pyblp.options.digits = 2
pyblp.options.verbose = False

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [2]:
cars = pd.read_csv('cars.csv') # this reads the *balanced* dataset (i.e. J = 40 products per market always)
# cars = pd.read_excel('cars.xlsx') # this reads the *unbalanced* dataset (i.e. J varies over time)

### No data for France pre 1990. Average growth in adult fraction from other countries applied each year before

In [3]:
AdultFrac = pd.read_excel("FracOver20.xlsx", index_col = 0)
cars['adults'] = None
for idx in cars.index:
    cars['adults'][idx] = AdultFrac[cars['ma'][idx]][cars['ye'][idx]]

C:\Users\PSand\AppData\Local\Temp\ipykernel_27920\553329577.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars['adults'][idx] = AdultFrac[cars['ma'][idx]][cars['ye'][idx]]


We estimate that 77% of the adult population have a driving license for a full-car. Hence, the share of population aged 20+ with a driver becomes:

In [4]:
license_share = 0.77
cars["ad_w_li"] = cars["adults"] * license_share 

In [5]:
lbl_vars = pd.read_csv('labels_variables.csv', index_col=0)
lbl_vals = pd.read_stata('cars.dta', iterator=True).value_labels() # the values that variables take (not relevant for all )

## Overview of the dataset

In [6]:
pd.set_option('display.max_colwidth', None)
tab = cars.mean(numeric_only=True).apply(lambda x: f'{x:.2f}').to_frame('Mean').join(lbl_vars)

# Set up for analysis

## Price variables 

Can be either price (`pr`), price-to-income (`princ`), or log price (`logp`, created below).

In [7]:
price_var = 'eurpr'

In [8]:
cars['logp'] = np.log(cars[price_var])

## Market share

In [9]:
# total quantity of cars sold in market-year (ma, ye)
cars['qu_tot'] = cars.groupby(['ma', 'ye'])['qu'].transform('sum')
cars['market_size'] = cars['pop'] * cars['ad_w_li']
cars['s'] = cars['qu'] / cars['market_size']

In [10]:
# compute the share of the outside good (will be useful for the demand inversion)
cars['s0'] = 1.0 - cars.groupby(['ma', 'ye'])['s'].transform('sum')
print(f'Outside share is from {cars.s0.min():.1%} to {cars.s0.max():.1%}')

Outside share is from 93.1% to 97.1%


In [11]:
cars.groupby(['ma'])['s'].describe().rename(index=lbl_vals['market']).style.format('{:.3f}')

,count,unique,top,freq
ma,,,,
Belgium,1200.000,1194.000,0.003,2.000
France,1200.000,1199.000,0.001,2.000
Germany,1200.000,1199.000,0.000,2.000
Italy,1200.000,1195.000,0.000,2.000
UK,1200.000,1199.000,0.000,2.000


## 1. Using canned software

In [12]:
from linearmodels.iv import IV2SLS
from statsmodels.api import OLS

In [13]:
cars['delta'] = cars['s'] / cars['s0']
cars['delta'] = np.log(cars['delta'].values.astype(float)) ## Den stoppede med at ville gøre det i et skridt uden at definere type

In [14]:
cars["brand"].replace('alfa romeo', 'alfa_romeo', inplace=True)
cars["brand"] = cars["brand"].str.replace('/', '', regex=False)

In [15]:
# Calculate the average price of all other cars in a given year in a given market:
# Step 1: Calculate the sum and count of prices for each year and market
cars['sum_eurpr_ye_ma'] = cars.groupby(['ye', 'ma'])['eurpr'].transform('sum')
cars['count_ye_ma'] = cars.groupby(['ye', 'ma'])['eurpr'].transform('count')

# Step 2: Calculate the average price excluding the current observation
cars['avg_eurpr_excl'] = (cars['sum_eurpr_ye_ma'] - cars['eurpr']) / (cars['count_ye_ma'] - 1)

# Drop the intermediate columns if they are no longer needed
cars.drop(columns=['sum_eurpr_ye_ma', 'count_ye_ma'], inplace=True)


cars['avg_eurpr_excl'] = np.log(cars['avg_eurpr_excl'])

In [16]:
# Make a copy of the dataframe for later making the IV-analysis
cars_iv = cars.copy()

In [17]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

In [18]:
print(f'The left out dummy for brands is: {dummies.columns[0]}')
print(f'There are: {len(dummies.columns)} brand-dummies')

The left out dummy for brands is: BMW
There are: 33 brand-dummies


In [19]:
# choose your preferred variables 
#x_vars = ['logp', 'avg_eurpr_excl', 'home', 'cy', 'hp', 'we', 'li', 'sp'] + x_vars_dummies # <--- !!! CHOOSE HERE 
x_vars = ['logp', 'home', 'cy', 'hp', 'we', 'li', 'sp'] + x_vars_dummies
print(x_vars)

['logp', 'home', 'cy', 'hp', 'we', 'li', 'sp', 'MCC', 'VW', 'alfa_romeo', 'audi', 'citroen', 'daewoo', 'daf', 'fiat', 'ford', 'honda', 'hyundai', 'innocenti', 'lancia', 'mazda', 'mercedes', 'mitsubishi', 'nissan', 'opel', 'peugeot', 'renault', 'rover', 'saab', 'seat', 'skoda', 'suzuki', 'talbot', 'talhillman', 'talmatra', 'talsimca', 'talsunb', 'toyota', 'volvo']


In [20]:
# set up the estimation equation
formula = 'delta ~ 1'
for x_ in x_vars:
    formula += ' + ' + x_
print(formula)

delta ~ 1 + logp + home + cy + hp + we + li + sp + MCC + VW + alfa_romeo + audi + citroen + daewoo + daf + fiat + ford + honda + hyundai + innocenti + lancia + mazda + mercedes + mitsubishi + nissan + opel + peugeot + renault + rover + saab + seat + skoda + suzuki + talbot + talhillman + talmatra + talsimca + talsunb + toyota + volvo


In [21]:
# Estimate the model by OLS
OLSmodel = OLS.from_formula(formula, cars).fit()
OLSmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  delta   R-squared:                       0.400
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     102.0
Date:                Mon, 21 Oct 2024   Prob (F-statistic):               0.00
Time:                        20:31:01   Log-Likelihood:                -6355.6
No. Observations:                5998   AIC:                         1.279e+04
Df Residuals:                    5958   BIC:                         1.306e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -8.8408      0.236    -37.483      0.000      -9.303      -8.378
logp           0.0479      0.032      1.499      0.134      -0.015       0.110
home           1.0292      0.022     47.423      0.000       0.987       1.072
cy            -0.0003   6.76e-05     -4.095      0.000      -0.000      -0.000
hp            -0.0317      0.002    -16.151      0.000      -0.036      -0.028
we             0.0005      0.000      3.834      0.000       0.000       0.001
li            -0.0264      0.011     -2.413      0.016      -0.048      -0.005
sp             0.0187      0.002     12.404      0.000       0.016       0.022
MCC           -1.0797      0.499     -2.166      0.030      -2.057      -0.102
VW             0.0961      0.061      1.582      0.114      -0.023       0.215
alfa_romeo    -0.5093      0.077     -6.598      0.000      -0.661      -0.358
audi          -0.1521      0.066     -2.296      0.022      -0.282      -0.022
citroen       -0.3539      0.059     -5.950      0.000      -0.470      -0.237
daewoo        -0.5044      0.290     -1.738      0.082      -1.073       0.064
daf           -0.7233      0.147     -4.931      0.000      -1.011      -0.436
fiat          -0.1852      0.058     -3.200      0.001      -0.299      -0.072
ford           0.1758      0.058      3.024      0.003       0.062       0.290
honda         -0.2551      0.086     -2.949      0.003      -0.425      -0.086
hyundai       -0.7687      0.253     -3.044      0.002      -1.264      -0.274
innocenti     -1.1701      0.189     -6.185      0.000      -1.541      -0.799
lancia        -0.8124      0.080    -10.197      0.000      -0.969      -0.656
mazda         -0.1721      0.080     -2.147      0.032      -0.329      -0.015
mercedes       0.3815      0.067      5.682      0.000       0.250       0.513
mitsubishi    -0.2566      0.112     -2.287      0.022      -0.476      -0.037
nissan        -0.1699      0.065     -2.597      0.009      -0.298      -0.042
opel           0.0323      0.059      0.552      0.581      -0.083       0.147
peugeot       -0.0966      0.059     -1.634      0.102      -0.213       0.019
renault        0.0066      0.057      0.115      0.909      -0.106       0.119
rover         -0.4077      0.065     -6.306      0.000      -0.534      -0.281
saab          -0.8801      0.270     -3.262      0.001      -1.409      -0.351
seat          -0.6178      0.091     -6.813      0.000      -0.796      -0.440
skoda         -0.7885      0.159     -4.974      0.000      -1.099      -0.478
suzuki        -0.7537      0.270     -2.792      0.005      -1.283      -0.225
talbot        -0.3994      0.081     -4.951      0.000      -0.557      -0.241
talhillman    -1.0643      0.229     -4.650      0.000      -1.513      -0.616
talmatra      -2.6303      0.500     -5.258      0.000      -3.611      -1.650
ta

In [22]:
test_formula = 'Intercept = 0, home = 0, cy = 0, hp = 0, we = 0, li = 0, sp = 0, MCC = 0, VW = 0, alfa_romeo = 0, audi = 0, citroen = 0, daewoo = 0, daf = 0, fiat = 0, ford = 0, honda = 0, hyundai = 0, innocenti = 0, lancia = 0, mazda = 0, mercedes = 0, mitsubishi = 0, nissan = 0, opel = 0, peugeot = 0, renault = 0, rover = 0, saab = 0, seat = 0, skoda = 0, suzuki = 0, talbot = 0, talhillman = 0, talmatra = 0, talsimca = 0, talsunb = 0, toyota = 0, volvo = 0'
OLSmodel.f_test(test_formula)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=196.4995562504705, p=0.0, df_denom=5.96e+03, df_num=39>

## Calculate elasticities

The Logit elasticities are 

$$
\mathcal{E}_{jk} \equiv \frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = 
\begin{cases}
\alpha (\mathbf{1}\{j = k\} - s_{jt}) p_{kt} & \text{if price is in level},   \\
\alpha (\mathbf{1}\{j = k\} - s_{jt})        & \text{if price is in log }. 
\end{cases}
$$

In [23]:
betaOLS = OLSmodel.params
elast_own = betaOLS['logp'] * (1 - cars['s'])
print(f'Price in logs:  Avg. own-price elasticity: {elast_own.mean(): .2%}')

elast_cross = - betaOLS['logp'] * cars['s']
print(f'Price in logs:  Avg. cross-price elasticity: {elast_cross.mean(): .2%}')

Price in logs:  Avg. own-price elasticity:  4.78%
Price in logs:  Avg. cross-price elasticity: -0.01%


$$
\frac{\partial s_{jt}}{\partial p_{kt}} = 
\begin{cases}
\alpha (\mathbf{1}\{j = k\} - s_{jt}) s_{jt} & \text{if price is in level},   \\
\alpha (\mathbf{1}\{j = k\} - s_{jt}) \frac{s_{jt}} {p_{kt}}      & \text{if price is in log }. 
\end{cases}
$$

In [24]:
cars['idx'] = cars.index

## Calculate the marginal cost for each car-type given firms are profitmaximizing

In [25]:
def MarginalCost(dat, index, beta, pvar, price, share, firm, log=True):
    p = dat[price].values
    firms = dat[firm].values
    H = (firms[:, None] == firms[None, :]).astype(np.int8)
    s = dat[share].values       # Column 's'
    alpha = beta[pvar]              # The given alpha value.

    # Compute the size of the matrix
    n = len(dat)

    # Create an identity matrix for the I(j = k) term
    #id_mat = np.eye(n)

    #divisor = 1
    #if log:
    #    divisor = p[:, None]

    # Compute s_dif matrix
    #s_dif = (alpha / divisor) * s[:, None] * (id_mat - s)

    s_dif = alpha * (np.diag(s) - np.outer(s, s))

    # Ensure H and s_dif are numeric arrays
    H = np.asarray(H, dtype=np.float64)
    s_dif = np.asarray(s_dif, dtype=np.float64)

    # Multiply H and s_dif element-wise
    Hs_dif = np.multiply(H, s_dif)

    # Ensure s_values is reshaped correctly and is a numeric array
    s = s.reshape((n, 1)).astype(np.float64)

    # Ensure p_values is also a numeric array
    p = np.asarray(p, dtype=np.float64)
    p = p.reshape((n, 1)).astype(np.float64)

    # Solve the system of equations hs_dif * c = s_values
    mc = p + np.linalg.inv(Hs_dif) @ s
    idx = dat[index].values
    idx = idx.reshape((n, 1)).astype(np.float64)
    return np.hstack((idx, mc))

In [26]:
def MC_loop(dat, index, beta, pvar, price, share, firm, market, year, log=True):

    mc_list = []

    for ye in dat[year].unique():
        for ma in dat[market].unique():
            sub_dat = dat[(dat[market] == ma) & (dat[year] == ye)].copy()
            mc = MarginalCost(sub_dat, index, beta, pvar, price, share, firm, log)
            mc_list.append(mc)

    # Vertically stack all arrays in the list into a single 2D array
    stacked_mc = np.vstack(mc_list)
    return stacked_mc

In [27]:
mc_array = MC_loop(cars, 'idx', betaOLS, 'logp', 'eurpr', 's', 'frm', 'ma', 'ye', log=True)  # mc_array[:, 0] is idx values, mc_array[:, 1] is mc values
mc_dict = dict(mc_array)
cars['mc_vanilla'] = cars['idx'].map(mc_dict)

In [28]:
def ccp(dat, beta, xvars, p, pvar, log): 
    '''
    INPUTS: 
        p: (J,) vector of prices
        t: (int) market index
    OUTPUTS:
        ccp: (J+1,) vector of conditional choice probabilities (0 = outside option)
    '''
    J = len(dat)
    assert p.shape == (J,)

    delta = np.zeros((J,))
    if 'Intercept' in beta:
        delta = np.ones((J,)) * beta['Intercept']

    for var in xvars:
        if var == pvar:
            continue
        delta += dat[var].values * beta[var]

    if log:
        delta += np.log(p) * beta[pvar]
    else:
        delta += p * beta[pvar]
    
    # 2. insert a zero in the first position for the outside option
    delta = np.insert(delta, 0, 0.0)
    
    # 3. max-rescale (to avoid numerical issues)
    delta -= delta.max() # no need for keepdims=True since delta.max() is a scalar

    # 4. compute the CCP
    ed = np.exp(delta)
    ccp = ed / ed.sum()

    return ccp # (J+1,)

In [29]:
def zeta(p, mc, H, dat, beta, xvars, pvar, log):
    J = len(p)
    assert (p.shape == (J,)) and (mc.shape == (J,)) and (H.shape == (J, J))
    s = ccp(dat, beta, xvars, p, pvar, log)
    s = s[1:] # remove outside option
    
    Lambda = beta[pvar] * np.diag(s)
    HLambda = np.multiply(H, Lambda)
    invHLambda = np.linalg.inv(HLambda)
    Gamma = beta[pvar] * np.outer(s, s)
    HGamma = np.multiply(H, Gamma)
    z = invHLambda @ (HGamma @ (p - mc) -  s)
    return z

In [30]:
def solve_nash_MS(p_start, mc, H, dat, xvars, pvar, beta, log=True, maxit=1000, tol=1e-6, DOPRINT=False): 
    p_prev = p_start.copy()
    for it in range(maxit): 
        p_next = mc + zeta(p_prev, mc, H, dat, beta, xvars, pvar, log)
        if np.linalg.norm(p_next - p_prev) < tol: 
            if DOPRINT: 
                print(f'Converged after {it} iterations')
            break 
        p_prev = p_next
    return p_next

In [31]:
def p_in_ye(dat, market, firm, mc_var, xvars, pvar, beta):
    p_list = []
    for ma in dat[market].unique():
        sub_dat = dat[(dat[market] == ma)].copy()
        firms = sub_dat[firm].values
        H = (firms[:, None] == firms[None, :]).astype(np.int8)
        mc = sub_dat[mc_var].values
        p_start = mc*1.5
        p = solve_nash_MS(p_start, mc, H, sub_dat, xvars, pvar, beta)
        p_list += [[p]]

    return p_list

In [32]:
cars['frm_m'] = cars['frm']
cars.loc[cars['frm_m'] == 4, 'frm_m'] = 26

In [33]:
dat99 = cars[(cars['ye'] == 99)].copy()
p_merger = p_in_ye(dat99, 'ma', 'frm_m', 'mc_vanilla', x_vars, 'logp', betaOLS)

In [34]:
print(p_merger[0])

[array([13664.1216386 , 24955.96734172, 12886.12315699,  5958.94384022,
        6072.94444648,  8103.5792121 , 10703.55089179,  8331.29361169,
       19554.94332661, 25585.37887349,  6756.22597238, 13967.82509347,
        9678.82118722,  6243.8599734 ,  9451.12413355,  7021.89655878,
        6319.71442987,  6528.51203825,  8727.95151125,  7422.37092497,
       10172.26789763, 14461.27180388, 22375.05403044, 17080.22004606,
       10437.08862403, 13189.71150018, 12430.65343043,  7572.18201942,
        9678.80134393,  5881.38243433, 13267.2745246 , 13379.30028553,
        8710.98643824,  6357.61290351, 11764.48043599, 10039.32432929,
        7135.8209109 , 12705.71811281,  7192.63949953, 11576.63215544])]


## IV-estimation

We use the Hausman-instrument. Start by computing this for the M-1 markets for each car.

In [35]:
# Ensure that the the country-column is treated as a string in the dataframe
cars_iv['ma'] = cars_iv['ma'].astype(str)

# Get the unique list of countries as strings
countries = cars_iv['ma'].unique()

# Step 2: Create the four additional columns for prices from the other four countries
for country in countries:
    # Create a new column for each country
    column_name = f'eurpr_in_{country}'
    
    # Create a copy of the dataframe that only includes the rows for the given country
    country_data = cars_iv[cars_iv['ma'] == country][['ye', 'type', 'eurpr']]
    
    # Rename the 'eurpr' column in this temporary dataframe to avoid confusion
    country_data = country_data.rename(columns={'eurpr': column_name})
    
    # Merge the country-specific prices back into the main dataframe on 'ye' and 'type'
    cars_iv = cars_iv.merge(country_data, on=['ye', 'type'], how='left')

# Step 4: For each row, remove the price of the current country from the new columns
for country in countries:
    column_name = f'eurpr_in_{country}'
    
    # Set the column to NaN where the country matches the current row
    cars_iv.loc[cars['ma'] == country, column_name] = None

In [36]:
price_columns = [f'eurpr_in_{country}' for country in countries]

In [37]:
# this is the Hausman-instrument we apply
cars_iv['avg_eurpr_other'] = cars_iv[price_columns].mean(axis=1)

In [38]:
cars_iv = cars_iv[cars_iv['avg_eurpr_other'].notna()] # drop NA observations for the instrument. 

In [39]:
# Define new dummies (this is why we made a new dataframe for the IV-estimation)
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars_iv[categorical_var]) # creates a matrix of dummies for each value of dummyvar
iv_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars_iv.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars_iv = pd.concat([cars_iv,dummies], axis=1)

In [40]:
# Define the exogenous variables in our model 
#x_vars_exog = ['avg_eurpr_excl', 'home', 'cy', 'hp', 'we', 'li', 'sp'] + iv_dummies

x_vars_exog = ['home', 'cy', 'hp', 'we', 'li', 'sp'] + iv_dummies

# Define the endogenous variables and their corresponding instruments
x_vars_endog = ['[logp ~ avdexr]']

In [41]:
# set up the estimation equation
iv_formula = 'delta ~ 1'
for x_ in x_vars_exog:
    iv_formula += ' + ' + x_
for x_ in x_vars_endog:
    iv_formula += ' + ' + x_


print(iv_formula)

delta ~ 1 + home + cy + hp + we + li + sp + MCC + VW + alfa_romeo + audi + citroen + daewoo + daf + fiat + ford + honda + hyundai + innocenti + lancia + mazda + mercedes + mitsubishi + nissan + opel + peugeot + renault + rover + saab + seat + skoda + suzuki + talbot + talhillman + talmatra + talsimca + talsunb + toyota + volvo + [logp ~ avdexr]


In [42]:
# Estimate the model by OLS
IVmodel = IV2SLS.from_formula(iv_formula, cars_iv).fit()
IVmodel.summary

c:\Users\PSand\anaconda3\lib\site-packages\linearmodels\iv\model.py:557: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                  delta   R-squared:                      0.3006
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2960
No. Observations:                5998   F-statistic:                 1.359e+05
Date:                Mon, Oct 21 2024   P-value (F-stat)                0.0000
Time:                        20:31:02   Distribution:                 chi2(39)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -3.4559     1.7002    -2.0327     0.0421     -6.7882     -0.1236
home           1.0914     0.0304     35.935     0.0000      1.0319      1.1509
cy            -0.0005     0.0001    -4.6584     0.0000     -0.0007     -0.0003
hp            -0.0378     0.0032    -11.854     0.0000     -0.0441     -0.0316
we             0.0018     0.0004     4.1698     0.0000      0.0009      0.0026
li            -0.1865     0.0514    -3.6311     0.0003     -0.2871     -0.0858
sp             0.0453     0.0086     5.2783     0.0000      0.0285      0.0621
MCC           -0.8340     0.1813    -4.5994     0.0000     -1.1894     -0.4786
VW             0.1743     0.0671     2.5972     0.0094      0.0428      0.3058
alfa_romeo    -0.7007     0.0959    -7.3038     0.0000     -0.8888     -0.5127
audi          -0.1889     0.0680    -2.7765     0.0055     -0.3222     -0.0555
citroen       -0.5172     0.0776    -6.6612     0.0000     -0.6694     -0.3650
daewoo        -0.3749     0.1761    -2.1284     0.0333     -0.7201     -0.0297
daf           -1.1673     0.2147    -5.4366     0.0000     -1.5881     -0.7465
fiat          -0.3477     0.0805    -4.3199     0.0000     -0.5054     -0.1899
ford           0.1650     0.0567     2.9093     0.0036      0.0539      0.2762
honda         -0.3130     0.0940    -3.3310     0.0009     -0.4972     -0.1288
hyundai       -0.9475     0.2084    -4.5476     0.0000     -1.3559     -0.5391
innocenti     -1.4927     0.1938    -7.7011     0.0000     -1.8725     -1.1128
lancia        -1.0288     0.1116    -9.2150     0.0000     -1.2476     -0.8100
mazda         -0.1989     0.0687    -2.8952     0.0038     -0.3335     -0.0642
mercedes       0.6807     0.1123     6.0638     0.0000      0.4607      0.9007
mitsubishi    -0.2226     0.0927    -2.4010     0.0164     -0.4043     -0.0409
nissan        -0.2135     0.0625    -3.4149     0.0006     -0.3361     -0.0910
opel          -0.0562     0.0663    -0.8475     0.3967     -0.1860      0.0737
peugeot       -0.1939     0.0698    -2.7771     0.0055     -0.3308     -0.0571
renault       -0.0752     0.0626    -1.2000     0.2301     -0.1979      0.0476
rover         -0.4187     0.0634    -6.5987     0.0000     -0.5430     -0.2943
saab          -1.3385     0.2168    -6.1744     0.0000     -1.7634     -0.9136
seat          -0.5439     0.0766    -7.0980     0.0000     -0.6941     -0.3937
skoda         -0.6077     0.1142    -5.3216     0.0000     -0.8315     -0.3838
suzuki        -0.7299     0.0880    -8.2941     0.0000     -0.9024     -0.5574
talbot        -0.6768     0.1200    -5.6416     0.0000     -0.9119     -0.4417
talhillman    -1.6362     0.3439    -4.7577     0.0000     -2.3102     -0.9621
talmatra      -3.1896     0.1932    -16.506     0.0000     -3.5683     -2.8109
talsimca      -0.8555     0.2099    -4.0754     0.0000     -1.2670     -0.4441
talsunb       -5.1290     0.3063    -16.744     0.0000     -5.7294     -4.5

In [43]:
beta_iv = IVmodel.params
elast_own_iv = beta_iv['logp'] * (1 - cars['s'])
print(f'Price in logs:  Avg. own-price elasticity: {elast_own_iv.mean(): .2%}')

elast_cross_iv = - beta_iv['logp'] * cars['s']
print(f'Price in logs:  Avg. cross-price elasticity: {elast_cross_iv.mean(): .2%}')

Price in logs:  Avg. own-price elasticity: -95.61%
Price in logs:  Avg. cross-price elasticity:  0.12%


In [44]:
mc_array = MC_loop(cars, 'idx', beta_iv, 'logp', 'eurpr', 's', 'frm', 'ma', 'ye', log=True)  # mc_array[:, 0] is idx values, mc_array[:, 1] is mc values
mc_dict = dict(mc_array)
cars['mc_iv'] = cars['idx'].map(mc_dict)

In [45]:
iv_xvars = x_vars_exog + ['logp']
dat99 = cars[(cars['ye'] == 99)].copy()
p_merger_iv = p_in_ye(dat99, 'ma', 'frm_m', 'mc_iv', x_vars, 'logp', beta_iv)

In [46]:
p_merger_iv[0]

[array([13664.08209663, 24955.92779975, 12885.9832499 ,  5959.06525625,
         6072.92589449,  8103.56066012, 10703.53233981,  8331.29497165,
        19554.93499286, 25585.37053973,  6756.12564444, 13967.72476553,
         9678.72085928,  6243.7200663 ,  9450.98422646,  7021.8172097 ,
         6319.63508079,  6528.39060856,  8727.9327994 ,  7422.24949528,
        10172.14646793, 14461.15037418, 22374.93260075, 17080.09861637,
        10437.08176659, 13189.63215111, 12430.51352334,  7572.17971495,
         9678.72199486,  5881.24252724, 13267.25597262, 13379.42170156,
         8710.84653115,  6357.59435153, 11764.47210223, 10039.30577731,
         7135.6810038 , 12705.69940096,  7192.62078768, 11576.51072575])]